In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
# import os,sys
# from PIL import Image

from helpers import *
from helpers_NN import *

%load_ext autoreload
%autoreload 2

import pip

try: 
    import cv2
except: 
    pip.main(['install', 'opencv-python'])
    import cv2 

try:
    import tensorflow as tf
except: 
    pip.main(['install', 'tensorflow'])
    import tensorflow as tf
print(tf.__version__)

1.4.0


In [ ]:
TRAINING_SIZE = 100

In [ ]:
data_dir = 'training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

# Extract it into numpy arrays.
all_train_data = extract_data_train(train_data_filename, TRAINING_SIZE)
all_train_labels = extract_labels(train_labels_filename, TRAINING_SIZE, train = True)

Loading training/images/satImage_001.png
Loading training/images/satImage_002.png
Loading training/images/satImage_003.png
Loading training/images/satImage_004.png
Loading training/images/satImage_005.png
Loading training/images/satImage_006.png
Loading training/images/satImage_007.png
Loading training/images/satImage_008.png
Loading training/images/satImage_009.png
Loading training/images/satImage_010.png
Loading training/images/satImage_011.png
Loading training/images/satImage_012.png
Loading training/images/satImage_013.png
Loading training/images/satImage_014.png
Loading training/images/satImage_015.png
Loading training/images/satImage_016.png
Loading training/images/satImage_017.png
Loading training/images/satImage_018.png
Loading training/images/satImage_019.png
Loading training/images/satImage_020.png
Loading training/images/satImage_021.png
Loading training/images/satImage_022.png
Loading training/images/satImage_023.png
Loading training/images/satImage_024.png
Loading training

In [ ]:
print(all_train_data.shape)
print(all_train_labels.shape)

In [ ]:
# Randomize data before using them
seed = 1
np.random.seed(seed)
all_train_data = np.random.permutation(all_train_data)
all_train_labels = np.random.permutation(all_train_labels)

In [ ]:
c0, c1 = 0, 0
for i in range(len(all_train_labels)):
    if all_train_labels[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print ('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

In [8]:
print ('Balancing training data...')
min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(all_train_labels) if j[0] == 1]
idx1 = [i for i, j in enumerate(all_train_labels) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]
print (len(new_indices))
print (all_train_data.shape)
all_train_data = all_train_data[new_indices,:]
all_train_labels = all_train_labels[new_indices]


train_size = all_train_labels.shape[0]

c0, c1 = 0, 0
for i in range(len(all_train_labels)):
    if all_train_labels[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print ('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Balancing training data...
2050112
(4000000, 10)
Number of data points per class: c0 = 1025056 c1 = 1025056


In [9]:
p = np.random.permutation(train_size)
all_train_data = all_train_data[p,:]
all_train_labels = all_train_labels[p,:]

In [10]:
# Neural network parameters
nb_hidden_nodes_1 = 500
nb_hidden_nodes_2 = 500
NUM_HIDDEN_LAYERS = 1
NUM_PATCH = train_size
NUM_FEATURES = all_train_data.shape[1]
NUM_LABELS = all_train_labels.shape[1]
BATCH_SIZE = 16 # 64
NUM_STEPS = 3001
NUM_EPOCHS = 1

In [11]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 100)

for ind, [train_index, test_index] in enumerate(kf.split(all_train_data)):
    data_train_cv, data_test_cv = all_train_data[train_index], all_train_data[test_index]
    labels_train_cv, labels_test_cv = all_train_labels[train_index], all_train_labels[test_index]
    
train_dataset = data_train_cv
train_labels = labels_train_cv

valid_dataset = data_test_cv
valid_labels = labels_test_cv


In [12]:
# train_dataset = all_train_data
# train_labels = all_train_labels

# valid_dataset = data_test_cv
# valid_labels = labels_test_cv

In [13]:
# Data to evaluate
root_testdir = "test_set_images"
test_names = os.listdir(root_testdir)

IMG_TEST_SIZE = len(test_names)

# Extract it into numpy arrays.
test_dataset = extract_data_test(root_testdir, IMG_TEST_SIZE)

Loading test_set_images/test_1/test_1.png
Loading test_set_images/test_10/test_10.png
Loading test_set_images/test_11/test_11.png
Loading test_set_images/test_12/test_12.png
Loading test_set_images/test_13/test_13.png
Loading test_set_images/test_14/test_14.png
Loading test_set_images/test_15/test_15.png
Loading test_set_images/test_16/test_16.png
Loading test_set_images/test_17/test_17.png
Loading test_set_images/test_18/test_18.png
Loading test_set_images/test_19/test_19.png
Loading test_set_images/test_2/test_2.png
Loading test_set_images/test_20/test_20.png
Loading test_set_images/test_21/test_21.png
Loading test_set_images/test_22/test_22.png
Loading test_set_images/test_23/test_23.png
Loading test_set_images/test_24/test_24.png
Loading test_set_images/test_25/test_25.png
Loading test_set_images/test_26/test_26.png
Loading test_set_images/test_27/test_27.png
Loading test_set_images/test_28/test_28.png
Loading test_set_images/test_29/test_29.png
Loading test_set_images/test_3/test_

In [14]:
print(valid_dataset.shape)
print(train_dataset.shape)
print(test_dataset.shape)

(20501, 10)
(2029611, 10)
(72200, 10)


In [15]:
def accuracy(predictions, labels):
    """ Performance measures of the network
        Computation of :
        - Accuracy
        - Precision (PPV)
        - Sensitivity (TPR)
        - Confusion matrix
    """
    # Accuracy
    accuracy = (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

    return accuracy

In [16]:
graph = tf.Graph()
with graph.as_default():

    # Input data.
    with tf.name_scope('Inputs_management'):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(BATCH_SIZE, NUM_FEATURES), name='tf_train_dataset')
        tf_train_labels = tf.placeholder(tf.int32, shape=(BATCH_SIZE, NUM_LABELS), name='tf_train_labels')

        tf_valid_dataset = tf.constant(valid_dataset, name="tf_valid_dataset")
        tf_test_dataset = tf.constant(test_dataset)

        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # Weights and bias.
    with tf.name_scope('Weights_bias_management'):                              
        def weight_variable(shape, name=None):
            """Create a weight variable with appropriate initialization."""
            return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name = name)


        def bias_variable(shape, name=None):
            """Create a bias variable with appropriate initialization."""
            return tf.Variable(tf.constant(0.1, shape=shape),name = name)

        if NUM_HIDDEN_LAYERS == 1:
             # Weights and bias for the first layer
            W_fc1 = weight_variable([NUM_FEATURES, nb_hidden_nodes_1], "W_fc1")
            b_fc1 = bias_variable([nb_hidden_nodes_1],"b_fc1")

            # Weights and bias for the second layer
            W_fc2 = weight_variable([nb_hidden_nodes_1, NUM_LABELS], "W_fc2")
            b_fc2 = bias_variable([NUM_LABELS],"b_fc2")

        elif NUM_HIDDEN_LAYERS == 2:
            # Weights and bias for the first layer
            W_fc1 = weight_variable([NUM_FEATURES, nb_hidden_nodes_1], "W_fc1")
            b_fc1 = bias_variable([nb_hidden_nodes_1],"b_fc1")

            # Weights and bias for the second layer
            W_fc2 = weight_variable([nb_hidden_nodes_1, nb_hidden_nodes_2], "W_fc2")
            b_fc2 = bias_variable([nb_hidden_nodes_2],"b_fc2")

            # Weights and bias for the third layer
            W_fc3 = weight_variable([nb_hidden_nodes_2, NUM_LABELS], "W_fc3")
            b_fc3 = bias_variable([NUM_LABELS],"b_fc3")

    
    # Model.
    def model(data):
        if NUM_HIDDEN_LAYERS == 1:
            # First layer
            with tf.name_scope('FullyConnected1'):
                h_fc1 = tf.matmul(data, W_fc1) + b_fc1
                h_relu1 = tf.nn.relu(h_fc1)

            # Second Layre
            with tf.name_scope('FullyConnected2'):
                h_fc2 = tf.matmul(h_relu1, W_fc2) + b_fc2
                valren = h_fc2

        elif NUM_HIDDEN_LAYERS == 2:
            # First layer
            with tf.name_scope('FullyConnected1'):
                h_fc1 = tf.matmul(data, W_fc1) + b_fc1
                h_relu1 = tf.nn.relu(h_fc1)

            # Second Layre
            with tf.name_scope('FullyConnected2'):
                h_fc2 = tf.matmul(h_relu1, W_fc2) + b_fc2
                h_relu2 = tf.nn.relu(h_fc2)

            # third Layer
            with tf.name_scope('FullyConnected3'):
                h_fc3 = tf.matmul(h_relu2, W_fc3) + b_fc3
                valren = h_fc3

        return valren 

    # Weights and bias.
    with tf.name_scope('Training_computations'):      
        logits = model(tf_train_dataset)
    
    with tf.name_scope('Loss_computation'):
        def compute_loss(logits, tf_train_labels, lambda_reg = 5e-4, regularization = True):
            if not regularization:
                loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
            else:
                reg = lambda_reg * (tf.nn.l2_loss(W_fc1) + tf.nn.l2_loss(W_fc2))
                loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels =  tf_train_labels) + reg)
            return loss

        loss = compute_loss(logits, tf_train_labels)


    # Optimizer.
    with tf.name_scope('Optimization'):
        
        # Optimizer: set up a variable that's incremented once per batch and
        # controls the learning rate decay.
        batch = tf.Variable(0)
        # Decay once per epoch, using an exponential schedule starting at 0.01.
        learning_rate = tf.train.exponential_decay(
            0.01,                # Base learning rate.
            batch * BATCH_SIZE,  # Current index into the dataset.
            train_size,          # Decay step.
            0.95,                # Decay rate.
            staircase=True)
        tf.summary.scalar('learning_rate', learning_rate)

        # Use simple momentum for the optimization.
#         optimizer = tf.train.MomentumOptimizer(learning_rate, 0.0).minimize(loss, global_step=batch)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    tf.summary.scalar("Loss", loss)
    summary_op = tf.summary.merge_all()
#     saver = tf.train.Saver(weightsDict)


    with tf.name_scope('Predictions'):
        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset), name="valid_prediction")

#         data_pred = tf.nn.softmax(classifier.model(tf_data, weightsDict)[0], name="output")
        test_prediction = tf.nn.softmax(model(tf_test_dataset)) 




with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized!')
    for epoch in range(0, NUM_EPOCHS):
        print("EPOCH :: %d" % (epoch))
        for step in range(NUM_STEPS):
            offset = (step * BATCH_SIZE) % (train_dataset.shape[0] - BATCH_SIZE)

            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + BATCH_SIZE), :]
            batch_labels = train_labels[offset:(offset + BATCH_SIZE), :]

            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.7}
            _, l, predictions, summary = session.run([optimizer, loss, train_prediction, summary_op], feed_dict=feed_dict)

            if step % 100 == 0:
                print('\nMinibatch loss at step %d: %f' % (step, l))

                print("Minibatch accuracy: %.1f%%" % accuracy(labels = batch_labels, predictions = predictions))
                print("Validation accuracy: %.1f%%" % accuracy(labels = valid_labels, predictions = valid_prediction.eval(feed_dict = {keep_prob:1.0})))
            
            
    test_pred = test_prediction.eval()
    print("\nTest prediction done!")
#     print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_dataset))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized!
EPOCH :: 0

Minibatch loss at step 0: 1.224394
Minibatch accuracy: 43.8%
Validation accuracy: 50.1%

Minibatch loss at step 100: 0.709770
Minibatch accuracy: 43.8%
Validation accuracy: 49.9%

Minibatch loss at step 200: 0.896669
Minibatch accuracy: 37.5%
Validation accuracy: 49.9%

Minibatch loss at step 300: 0.722460
Minibatch accuracy: 50.0%
Validation accuracy: 50.4%

Minibatch loss at step 400: 0.715565
Minibatch accuracy: 50.0%
Validation accuracy: 50.1%

Minibatch loss at step 500: 0.701525
Minibatch accuracy: 50.0%
Validation accuracy: 49.8%

Minibatch loss at step 600: 0.675718
Minibatch accuracy: 62.5%
Validation accuracy: 49.9%

Minibatch loss at step 700: 0.677330
Minibatch accuracy: 62.5%
Validation accuracy: 49.8%

Minibatch loss at step 800: 0.666614
Minibatch accuracy: 56.2%
Validation accuracy: 49.6%

Minibatch loss at step 900: 0.664855
Minibatch accuracy: 68.8%
Validation accuracy: 

In [17]:
y_pred = [np.argmax(p) for p in test_pred]
create_submission(y_pred, "submission_NN.csv")